# Singular Value Decomposition (SVD)

Du hast jetzt bereits NMF und LDA zum Topic Modeling kennengelernt. Ein anderes Verfahren aus der linearen Algebra ist die sog. *Hauptachsentranformation* und davon abgeleitet die *Singulärwertzerlegung*. Da diese vielfältig eingesetzt werden kann, schauen wir uns die auch noch an und bewerten abschließend die unterschiedlichen Verfahren.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Anschließend führst du die Vektorisierung durch. Das kann auch wieder einen Augenblick dauern:

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5, use_idf=False)
tfidf_vectors = tfidf_vectorizer.fit_transform(df["nav"])
tfidf_vectors

Durch die Nutzung von `min_df=5` ist die Matrix einigermaßen übersichtlich geblieben!

## Topic Model mit SVD berechnen

Die Aufrufsyntax des Topic Models ist sehr ähnlich zu der des `TfidfVectorizers`, nur dass du hier keine Transformation durchführen musst, daher heißt die Methode nur `fit`. Der Aufruf kann ein paar Sekunden dauern:

In [ ]:
from sklearn.decomposition import TruncatedSVD

num_topics = 10

svd = TruncatedSVD(n_components = num_topics)
svd.fit(tfidf_vectors)

Wie gewohnt zeigst du die Topics an:

In [ ]:
def topics_table(model, feature_names, n_top_words = 20):
    word_dict = {}
    
    for i in range(model.n_components):
        # ermittle für jedes Topic die größten Werte
        words_ids = model.components_[i].argsort()[:-n_top_words-1:-1]
        words = [feature_names[key] for key in words_ids]
        # und füge die entsprechenden Worte im Klartext dem Dictionary hinzu
        word_dict['Topic #%02d' % i] = words;
    
    return pd.DataFrame(word_dict)

In [ ]:
topics_table(svd, tfidf_vectorizer.get_feature_names())

Das Ergebnis sieht schon ziemlich gut aus. Abgesehen von dem ersten Topic (#00) sind alle ziemlich gut interpretierbar! Im Gegensatz zu den anderen Verfahren unterscheiden sich die wichtigsten Wörter der Topics häufig nicht.

In [ ]:
svd.singular_values_ / svd.singular_values_.sum() * 100.0

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def wordcloud_topic_model_summary(model, feature_names, no_top_words):
    for topic in model.components_:
        freq = { feature_names[i].replace(" ", "_"): topic[i] for i in topic.argsort()[:-no_top_words - 1:-1]}
        wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
        wc.generate_from_frequencies(freq)
        plt.figure(figsize=(12,12))
        plt.imshow(wc, interpolation='bilinear')

Die weniger differenzierten Topics siehst du auch in den Wordclouds:

In [ ]:
wordcloud_topic_model_summary(svd, tfidf_vectorizer.get_feature_names(), 40)

## NMF, LDA oder lieber doch SVD?

So berechtigt diese Frage ist, so schwer ist sie zu beantworten. In den allermeisten Fällen wird NMF schon ziemlich gute Resultate liefern. SVD ist hauptsächlich aus konzeptionellen Gründen und zur Dimensionsreduktion interessant. Bei LDA ist es schwer zu entscheiden, ob der deutlich höhere Rechenaufwand durch das Sampling wirklich gerechtfertig ist.

Du kannst dich auch zwischen `scikit-learn` und `gensim` entscheiden. Während ersteres das bessere API hat und sich leichter auch für überwachte Lernverfahren einsetzen lässt, kann letzteres mehr Methoden und insbesondere auch *Scores* ausrechnen, leidet aber etwas unter seinem API. Es schadet keinesfalls, wenn du dich mit beidem vertraut machst!

## Topic Models werden oft vergessen

Bei unüberwachten Lernverfahren denkt man meistens an *Clustering*, was aber für (lange) Texte nur eingeschränkt einsatzfähig ist. Topic Models sind häufig schneller zu berechnen und du solltest sie unbedingt in dein Analyse-Portfolio aufnehmen.